In [20]:
import os

os.environ["SEED"] = "0"

import numpy as np

from src.aac.AAC import AAC
from src.constant import DATA_DIR, DATABASE_DIR, MAIN_DIR, SEED
from src.database import DB
from src.database.queries import get_model_training_data, get_solvers_count
from src.experiment import parhydra
from src.instance.TSP_Instance import TSP_from_index_file, set_n22_cut_off_time
from src.log import logger
from src.solver.Portfolio import Portfolio
from src.solver.TSP_LKH_Solver import TSP_LKH_Solver
from src.surrogate.SurrogatePolicy import (
    EvaluationSurrogatePolicyA,
    EvaluationSurrogatePolicyB,
    EvaluationSurrogatePolicyC,
    IterationSurrogatePolicyA,
    IterationSurrogatePolicyB,
    SurrogatePolicy,
    TestSurrogatePolicy,
)
from src.surrogate.wrapper import EmptyWrapper, SurvivalFunctionWrapper

In [21]:
train_instances = TSP_from_index_file(
    filepath=DATA_DIR / "TSP" / "TRAIN" / "index.json",
    cut_off_cost=100,
    cut_off_time=10,
    n=5,
    seed=SEED
)
test_instances = TSP_from_index_file(
    filepath=DATA_DIR / "TSP" / "TEST" / "index.json",
    cut_off_cost=1000,
    cut_off_time=100,
    n=250,
    seed=0
)
test_instances

InstanceList(size=250)[TSP_Instance(filepath=TSP/TEST/cluster/150.tsp), TSP_Instance(filepath=TSP/TEST/cluster/170.tsp), TSP_Instance(filepath=TSP/TEST/cluster/000.tsp), TSP_Instance(filepath=TSP/TEST/cluster/090.tsp), TSP_Instance(filepath=TSP/TEST/cluster/094.tsp), TSP_Instance(filepath=TSP/TEST/cluster/013.tsp), TSP_Instance(filepath=TSP/TEST/cluster/003.tsp), TSP_Instance(filepath=TSP/TEST/cluster/182.tsp), TSP_Instance(filepath=TSP/TEST/cluster/054.tsp), TSP_Instance(filepath=TSP/TEST/cluster/120.tsp), TSP_Instance(filepath=TSP/TEST/cluster/055.tsp), TSP_Instance(filepath=TSP/TEST/cluster/007.tsp), TSP_Instance(filepath=TSP/TEST/cluster/108.tsp), TSP_Instance(filepath=TSP/TEST/cluster/104.tsp), TSP_Instance(filepath=TSP/TEST/cluster/032.tsp), TSP_Instance(filepath=TSP/TEST/cluster/132.tsp), TSP_Instance(filepath=TSP/TEST/cluster/184.tsp), TSP_Instance(filepath=TSP/TEST/cluster/112.tsp), TSP_Instance(filepath=TSP/TEST/cluster/048.tsp), TSP_Instance(filepath=TSP/TEST/cluster/078.tsp

In [ ]:
train_instances = set_n22_cut_off_time(train_instances, reference_cut_off_time=10.0)

In [ ]:
surrogate_policy = TestSurrogatePolicy(
    first_fit_solver_count=5,
    refit_solver_count=2,
)

surrogate_policy = EvaluationSurrogatePolicyA(
    first_fit_solver_count=5,
    refit_solver_count=2,
    pct_chance=0.5,
)

# surrogate_policy = EvaluationSurrogatePolicyB(
#     first_fit_solver_count=5,
#     refit_solver_count=2,
#     reevaluate_pct=0.5,
# )

# surrogate_policy = EvaluationSurrogatePolicyC(
#     first_fit_solver_count=5,
#     refit_solver_count=2,
#     reevaluate_factor=1.0,
# )

# surrogate_policy = IterationSurrogatePolicyA(
#     first_fit_solver_count=5,
#     refit_solver_count=2,
#     iter_diff=2,
# )

# surrogate_policy = IterationSurrogatePolicyB(
#     first_fit_solver_count=5,
#     refit_solver_count=2,
# )

In [ ]:
SOLVERS_N = 1
ATTEMPTS_N = 1
MAX_ITER = 30

portfolio = parhydra(
    train_instances=train_instances,
    surrogate_policy=surrogate_policy,
    SOLVERS_N=SOLVERS_N,
    ATTEMPTS_N=ATTEMPTS_N,
    MAX_ITER=MAX_ITER,
)